# Implementing Latent Dirichlet Allocation for Topic Modeling and Document Classification

### Andrew Cooper, Brian Cozzi

## Abstract

In this report we implement a form of Latent Dirichlet Allocation for topic modeling. Latent Dirichlet Allocation (LDA) was first introduced by Blei, Ng, and Jordan in 2003 as a hierarchical modeling approach for discrete data such as text in a corpus. This algorithm hinges on the notion that collections of data, such as text in a document, are generated according to a latent topic distribution, where each topic assigns probabilities to different words. The purpose of LDA in topic modeling is to group documents based on similar topic distributions, and to identify key words in each topic. Using a collapsed Gibbs sampler approach to LDA as described in Darling 2011, we implement an algorithm that estimates the latent topic distributions of a given corpus of documents. In addition, our algorithm returns key words assigned to each topic. We evaluate the performance of our algorithm on both simulated data and documents from the Newsgroups corpus.

Key phrases: Latent Dirichlet Allocation, Topic Modeling, Collapsed Gibbs Sampler,

## 1. Background (Brian)

$\textbf{State the research paper you are using. Describe the concept of the algorithm and why it is interesting and/or useful. If appropriate, describe the mathematical basis of the algorithm.}$

### Topic Modeling
Across a variety of disciplines, the simplification of large sets of categorical data are necessary to quantify similarities, detect anomialies between these large and superficially heterogeneous THINGS. 
- Applications of LDA beyond text \

Latent Dirichlet Allocation (LDA), introduced by Blei, Ng, and Jordan in their seminal 2003 paper, develops an intuitive framework through which these types of problems may be viewed. This original paper primarily focused on topic modeling as it pertains to text data. For consistency and in the interest of comprehensibility, will focus the remainder of this paper on LDA's application to text data. 

### How does this model work?
The paper proposed a hierarchical Bayesian model for creating documents from words that are associated with a mixture of topics. 
- Definetti's theorem - exchangeability
- LDA as a Hierarchical model

### How can it be applied to our research
Topic modeling and LDA is everywhere there is text data. Topic modeling for health records

### Advantages over other algorithms
- LSI
- pLSI

### Disadvantages
- difficult to implement
- While it's not unique to this approach of topic modeling, it also neglects the word order. There are certain extensions of this problem to get around this. For example, rather than just considering occurrences of single words, we can also consider $n$ adjacent words (commonly called $n$-grams) to capture information about phrases. In this case too, the order of the $n$-grams is neglected.

### What will we do in this paper
In this paper, we will describe our approach to implement LDA using a collapsed Gibbs sampler method described in Darling 2011. Section 2 will discuss the algorithm that has been implemented while Section 3 will provide an overview of how this algorithm was optimized. Sections 4 and 5 will demonstrate this algorithms application to simulated and  real-world datasets respectively. Section 6 will compare this algorithm to others $\textbf{SPECIFY THIS LATER}$. We will conclude with a discussion in section 7.

## 2. Description of Algorithm

The LDA algorithm takes in 4 inputs: a corpus of documents, the number of topics, two optional choices for hyperparameters, and an optional specification on the number iterations for the Gibbs Sampler. The ultimate goal of the algorithm is to estimate the topic distribution for each document as well as the word distribution for each topic. It does this by making inference on the latent topics of each word in the given corpus. We perform this inference by implementing a Gibbs sampler. First the algorithm sets a starting point by randomly sampling topics for each of the words in the corpus. Then it iteratively samples new topics for each word using calculated posterior probabilities. After a number of iterations, the Gibbs Sampler returns the estimated topics for each word, which are then used to calculate the latent topic distributions and word distributions using Monte Carlo estimation. These estimated quantities are returned to the user.

### 2.1 Variables

The LDA algorithm has many different symbols and components. We establish all the symbols used in our algorithm below:

* $K$ = The number of topics
* $M$ = The number of documents in the corpus
* $N_m$ = The number of words in document $m$
* $V$ = The number of possible words
* $w_{m,n}$ = The nth word in document $m$
* $z_{m,n}$ = The nth topic in document $m$ 
* $\theta_m$ = The topic distribution of document $m$
* $\phi_k$ = The word distribution of topic $k$

### 2.2 Algorithm Input

The algorithm takes as input a corpus of documents represented as a $M$x$V$  bag-of-words matrix. In addition, it takes as input the number of topics $K$. It also takes in two positive values representing the hyperparameters for the topic distribution ($\alpha$) and the word distribution ($\beta$). For this implementation we use symmetric priors for the dirichlet distributions, which means only one value is needed as input for each prior. Finally it takes as input the number iterations for the Gibbs sampler.

### 2.3 Gibbs Sampler

The algorithm for LDA has three main steps. The first step of the algorithm is preparing the data for the Gibbs sampler. As a starting point for our sampler, we first must randomly assign topics $z_{m,n}$ for each of the words in the given corpus. We then create two different count matrices, $N_1$ and $N_2$: $N_1$ is a $M$x$K$ matrix that counts the distribution of topics across documents. $N_2$ is a $K$x$V$ matrix that counts the distribution of words across topics. The count matrices are initialized according to the random topic assignment.

The second step is the implementation of the Gibbs sampler. For each iteration, our sampler loops through every word $w_{m,n}$ in every document of our corpus. For each word, we first remove its assigned topic and decrement the appropriate count matrices $N_1$, $N_2$, and $N_3$. We then calculate the posterior probabilities of the word belonging to each of the possible topics. One of these topics is sampled using these probabilities and is assigned to the word. Finally, all of the count matrices are incremented according to this new topic. This process is done for all the words in the corpus for how many iterations the user specified.

***Gibbs Sampler Implementation***

* Randomly assign topics $z_{m,n}$ for all words in corpus, and initialize count matrices $N_1$ and $N_2$

* **for** $i = 1$ to n_iter:
   * **for** $m = 1$ to $M$:
       * **for** $n = 1$ to $N_m$:
           * $w = w_{m,n}$
           * $z_{old} = z_{m,n}$
           * $N_1[m, z_{old}] -= 1$
           * $N_2[z_{old}, w] -= 1$
           * **for** $k = 1$ to $K$:
               * $p_k = Pr(z = k | \dots) \propto (N_1[m, k] + \alpha[k])*\dfrac{N_2[k, w] + \beta[w]}{\sum_V{N_2[k, v] + \beta[v]}}$
           * Normalize $p$ 
           * $z_{new} = $ Sample from $Cat(K, p)$
           * $z_{m,n} = z_{new}$
           * $N_1[m, z_{new}] += 1$
           * $N_2[z_{new}, w] += 1$

### 2.4 Parameter Estimation

The third step to the algorithm is estimating and returning the quantities of interest. One quantity to estimate is the topic distribution $\theta_m$ for each document $m$. Another quantity to estimate is the word distribution $\phi_k$ for each topic. These quantities are estimated using the count matrices $N_1$ and $N_2$ established in the Gibbs sampler:

$\hat{\theta}_{m,k} = \dfrac{N_1[m, k] + \alpha[k]}{\sum_k{N_1[m, k] + \alpha[k]}}$ 

$\hat{\phi}_{k, v} = \dfrac{N_2[k, v] + \beta[v]}{\sum_v{N_2[k, v] + \beta[v]}}$

## 3. Description of Performance Optimization

In the algorithm described above, we see that each iteration of the algorithm runs in $O(MNK)$ time. On its surface, this does not appear to scale well especially as more documents are added to the corpus. However, leveraging Python's interface with C and its ability to execute some loops in parallel, we can make the run time much more managable.

In [2]:
### Parallel execution with Cython

#Will not work unless OpenMP is installed.

%%cython --compile-args=-fopenmp --link-args=-fopenmp --force

import cython
from cython.parallel import parallel, prange

@cython.boundscheck(False)
@cython.wraparound(False)
def matrix_multiply2(double[:,:] u, double[:, :] v, double[:, :] res):
    cdef int i, j, k
    cdef int m, n, p

    m = u.shape[0]
    n = u.shape[1]
    p = v.shape[1]

    with cython.nogil, parallel():
        for i in prange(m):
            for j in prange(p):
                res[i,j] = 0
                for k in range(n):
                    res[i,j] += u[i,k] * v[k,j]

SyntaxError: invalid syntax (<ipython-input-2-f1d107f99ac4>, line 5)

In [1]:
#Parallelization with vectorize and guvectorize
#----

@numba.vectorize([float64(float64, float64),
                  float32(float32, float32),
                  float64(int64, int64),
                  float32(int32, int32)],
                 target='parallel')
def f_parallel(x, y):
    return np.sqrt(x**2 + y**2)

SyntaxError: invalid syntax (<ipython-input-1-f3728385eed5>, line 1)

# In the next section we will show that this code optimization offers a considerable speed up over the standard Python implementation.

## 4. Applications to Simulated Datasets

## 5. Applications to Real Dataset

- topic distribution by document
- cluster topics (extra)

## 6. Comparative Analysis with Competing Algorithms

## 7. Discussion/Conclusion

## 8. Installation Guide

https://packaging.python.org/tutorials/packaging-projects/

## References/Bibliography

1) Darling, W.M. (2011). A Theoretical and Practical Implementation Tutorial on Topic Modeling and Gibbs Sampling.

2) David M. Blei, Andrew Y. Ng, and Michael I. Jordan. 2003. Latent dirichlet allocation. J. Mach. Learn. Res. 3 (March 2003), 993-1022.